In [53]:
import numpy as np
import gym
from tqdm import tqdm
import copy

In [150]:
#this class is the enviroment we use 
#the openAI enviornment hase some bug so we create this one 
class FrozenLake:
    def __init__(self):
        self.Grid=[['S','F','F','F'],['F','H','F','H'],['F','F','F','H'],['H','F','F','G']]
        self.pos=None
        self.N=4
    
    #start the game should be called after we create the object
    def reset(self):
        """
        this function start the game 
        reset the bored and the inite position
        return 0 wich is the inital state
        """
        self.pos=[0,0]
        self.grid=copy.deepcopy(self.Grid)
        self.grid[self.pos[0]][self.pos[1]]='*'
        return 0
    
    #to validate the move we wont make it 
    def _valid_move(self,x,y):
        """
        validation move from user 
        parameter (x,y):
            x:is index of row
            y is index of column
        return bool:
            True if can move to the cell[x][y]
            False other wise
        """
        if x<self.N and y<self.N and x>=0 and y>=0:return True
        else:return False
        
    #this one creat the step and return obs,reward,done
    def step(self,action):
        """
        this function take an action and reutun the state of game
        parameter:
            action=range(4)
            {0:Up,1:Down,2:right,3:left}
        return :(state,reward,done)
            state:the new state is in range [0,15]
            reward 0 if not finish -1 if he loss 1 if we reach the Gole
            done False while not (win or loss)
        """
        reward=-1
        done=False
        if action==0:
            #go Up
            if self._valid_move(self.pos[0]-1,self.pos[1]):
                if self.grid[self.pos[0]-1][self.pos[1]]=='H':
                    reward=-1
                    done=True
                elif self.grid[self.pos[0]-1][self.pos[1]]=='G':
                    reward=1
                    done=True
                else:
                    reward=0
                    done=False
                self.grid[self.pos[0]-1][self.pos[1]]='*'
                self.pos[0]-=1
        elif action==1:
            #go Down
            if self._valid_move(self.pos[0]+1,self.pos[1]):
                if self.grid[self.pos[0]+1][self.pos[1]]=='H':
                    reward=-1
                    done=True
                elif self.grid[self.pos[0]+1][self.pos[1]]=='G':
                    reward=1
                    done=True
                else:
                    reward=0
                    done=False
                self.grid[self.pos[0]+1][self.pos[1]]='*'
                self.pos[0]+=1
        elif action==2:
            #go Right
            if self._valid_move(self.pos[0],self.pos[1]+1):
                if self.grid[self.pos[0]][self.pos[1]+1]=='H':
                    reward=-1
                    done=True
                elif self.grid[self.pos[0]][self.pos[1]+1]=='G':
                    reward=1
                    done=True
                else:
                    reward=0
                    done=False
                self.grid[self.pos[0]][self.pos[1]+1]='*'
                self.pos[1]+=1
        elif action==3:
            #go left
            if self._valid_move(self.pos[0],self.pos[1]-1):
                if self.grid[self.pos[0]][self.pos[1]-1]=='H':
                    reward=-1
                    done=True
                elif self.grid[self.pos[0]][self.pos[1]-1]=='G':
                    reward=1
                    done=True
                else:
                    reward=0
                    done=False
                self.grid[self.pos[0]][self.pos[1]-1]='*'
                self.pos[1]-=1
        return (self.pos[0]*self.N+self.pos[1]),reward,done
    
    def render(self):
        """
        print the Bord of Game
        """
        for i in self.grid:
            print(i)

In [151]:
env=FrozenLake()

In [255]:
def exploration_policy(epsilon,state,q_tabel):
    """
    this function is for random action 
    we use epsilon policy for random actions selection
    return integer number from [0,3] which indecate the action 
    """
    if np.random.rand()<epsilon:
        return np.random.randint(1,4)
    else:
        return np.argmax(q_tabel[state])

    
#inite the Q_value Tabel where the row is the state and column is action we can do
Q_tabel=np.zeros((16,4))

#learning rate for Q_value algorithem 
lr=0.8

#the discount factor 
gamma=0.95

#epsilon for epsilon policy
epsilon=1

#number of epoch
epochs=1000

#number of step we can do in each game
num_steps=100

#the list for save the reward in each game to check how many game we have been win
reward_List=[]

for episod in tqdm(range(1,epochs)):
    #reset the game
    state=env.reset()
    #acumlater for reward in each game which is 0 or -1 or 1
    rewards=0
    for j in range(num_steps):
        #create an action 
        action=exploration_policy(epsilon,state,Q_tabel)
        
        #update epsilon which is decrease while we are learning 
        # and that becouse in start of traning we wont to explore the environment in random way
        # and when we learn somthing we wont to take information from what we learn 
        epsilon = max(1 - episod / 140, 0.1)
        
        #make move
        new_state,reword,done=env.step(action)
        
        #implement of the Q_value equation you can check in the obove comment
        Q_tabel[state,action]*=(1-lr)
        Q_tabel[state,action]+=lr*(reword+gamma*np.max(Q_tabel[new_state]))
        
        #update state and rewards
        state=new_state
        rewards+=reword
        
        #if finish the Game Break the loop and start new game 
        if done:
            break
    reward_List.append(rewards)

100%|██████████| 999/999 [00:00<00:00, 2285.61it/s]


In [256]:
sum(reward for reward in reward_List if reward==1)/1000

0.743

In [257]:
Q_tabel

array([[-0.8       ,  0.77378094,  0.77378094, -0.26490811],
       [-0.8       , -1.        ,  0.81450625,  0.73509189],
       [-0.8       ,  0.857375  ,  0.77378094,  0.77378094],
       [-0.8       , -0.999936  , -0.40560429,  0.81450625],
       [ 0.        ,  0.81450625, -1.        , -0.22621906],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.9025    , -1.        , -1.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        , -1.        ,  0.857375  , -0.18549375],
       [-0.8       ,  0.9025    ,  0.9025    ,  0.81450625],
       [ 0.        ,  0.95      , -1.        ,  0.857375  ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        , -0.99968   ,  0.95      , -0.9999872 ],
       [ 0.        , -0.05      ,  1.        ,  0.9025    ],
       [ 0.        ,  0.        ,  0.        ,  0.        ]])

In [260]:
def play_game(env,q_tabel):
    """
    this is to play a game depending on the Q_tabel that we learn
    """
    state=env.reset()
    for i in range(15):
        action=np.argmax(q_tabel[state,:])
        obs,reward,done=env.step(action)
        env.render()
        if done:
            if reward==1:
                print('win')
                break
            else:
                print('loss')
                break
        state=obs

In [261]:
play_game(env,Q_tabel)

['*', 'F', 'F', 'F']
['*', 'H', 'F', 'H']
['F', 'F', 'F', 'H']
['H', 'F', 'F', 'G']
['*', 'F', 'F', 'F']
['*', 'H', 'F', 'H']
['*', 'F', 'F', 'H']
['H', 'F', 'F', 'G']
['*', 'F', 'F', 'F']
['*', 'H', 'F', 'H']
['*', '*', 'F', 'H']
['H', 'F', 'F', 'G']
['*', 'F', 'F', 'F']
['*', 'H', 'F', 'H']
['*', '*', '*', 'H']
['H', 'F', 'F', 'G']
['*', 'F', 'F', 'F']
['*', 'H', 'F', 'H']
['*', '*', '*', 'H']
['H', 'F', '*', 'G']
['*', 'F', 'F', 'F']
['*', 'H', 'F', 'H']
['*', '*', '*', 'H']
['H', 'F', '*', '*']
win


# we will use openAI gym in to implement Q-learnig algorithem using 
# Taxi-v3

In [1]:
import numpy as np
import gym 
import random 
import tqdm as tqdm

In [2]:
env=gym.make('Taxi-v3')

## browse the environment 

In [4]:
print("The number of state is :",env.observation_space.n," The number of actions are :",env.action_space.n)

The number of state is : 500  The number of actions are : 6


In [25]:
Q_tabel=np.zeros((env.observation_space.n,env.action_space.n))
Q_tabel.shape

(500, 6)

## define the learnig parameter

In [8]:
#the number of episod (epoch)
episods=50000

#the number of steps in each episod 
steps=100

#this for epsilon-greedy policy
epsilon=1

#learning rate 
lr=0.7

#discount rate 
gamma=0.618

decay_rate=0.01
max_epsilon=1
min_epsilon=0.01

In [7]:
def epsilon_greedy_policy(epislon,state):
   
    if np.random.rand()<epsilon:
        return np.random.randint(0,5)
    else:
        return np.argmax(Q_tabel[state])

In [32]:
def Q_learning_algorithem(env,gamma=0.9,lr=0.8,epsilon=1,episods=20000,num_steps=200):
    
    reward_List=[]
    
    Q_tabel=np.zeros((env.observation_space.n,env.action_space.n))
    
    for episod in tqdm.tqdm(range(1,episods)):
        #reset the game
        state=env.reset()
        
        for j in range(num_steps):
            #create an action 
            action=epsilon_greedy_policy(epsilon,state)

            #update epsilon which is decrease while we are learning 
            # and that becouse in start of traning we wont to explore the environment in random way
            # and when we learn somthing we wont to take information from what we learn 

            #make move
            new_state,reword,done,info =env.step(action)

            #implement of the Q_value equation you can check in the obove comment
            Q_tabel[state,action]*=(1-lr)
            Q_tabel[state,action]+=lr*(reword+gamma*np.max(Q_tabel[new_state]))

            #update state and rewards
            state=new_state

            #if finish the Game Break the loop and start new game 
            if done:
                break
        reword=0 if reword<0 else 1        
        reward_List.append(reword)
        epsilon = max(1 - episod / 6000, 0.1)
    return reward_List,Q_tabel

In [33]:
reword_list,Q_tabel=Q_learning_algorithem(env)

100%|██████████| 19999/19999 [13:52<00:00, 24.03it/s]


In [34]:
np.sum(reword_list)/5000

0.0

In [35]:
def play_taxi_game(Q_tabel,env):
    state=env.reset()
    for i in range(25):
        action=np.argmax(Q_tabel[state])
        obs,reward,done,info=env.step(action)
        env.render()
        if done:
            if reward>0:
                print('win')
            else:
                print('loss')
            break
        state=obs

In [36]:
play_taxi_game(Q_tabel,env)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
+---------+
|R: | : :G|
| : | : : |
| : 